In [11]:
import numpy as np
import os
import glob
import pandas as pd

# Image processing tools
import skimage
import skimage.io
import skimage.filters
import skimage.morphology

# import bokeh
# import bokeh.io #for some reason have to separately import bokeh.io
# bokeh.io.output_notebook()

# rename folders to change _# into a well name

In [2]:
def get_tuples_of_interest (dirpath):
    
    #get only terminal directories that will have images in them, walk also reports the parent directory its told to walk in
    #and we want to get rid of that
    tuples_of_interest = []
    for entry in list(os.walk(dirpath)):
        #if there are no subdirectories in this folder (meaning its a terminal directory that has image files in it)
        if not entry[1]:
            #tuples contain [0] the path for the directory, [1] any subdirectories in this one, [2] files in this directory
            tuples_of_interest.append(entry)
            
    return tuples_of_interest

In [3]:
directory = 'Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190725 a=b big flask/raw images/'

tuples_of_interest = get_tuples_of_interest(directory)

In [4]:
len(tuples_of_interest)

96

In [11]:
tuples_of_interest[0][0].split('/')

['Z:',
 'Reed',
 'Projects',
 'micro_consortia',
 'DARPA_biocon',
 'Task 1.1',
 'A=B',
 '20190725 a=b big flask',
 '0_0-0_10000x_1']

In [76]:
#create non general dictionary to map '_#' suffixes on microscope picture folders to well names
#this will need to be updated when you have a new experiment with new well maps etc.

num = ['1', '2', '3', '4']
let = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
wells = [l+n for l in let for n in num]
suff = [str(x) for x in list(range(1,len(wells)+1))]

rename = {s:w for s,w in zip(suff, wells)}

In [77]:
directories = []

#this step needs to be updated when the length of the time string changes

for t in tuples_of_interest:
#     print(t[0].split('/')[-1][:4])
    if t[0].split('/')[-1][:4] == time:
        directories.append(t[0])

In [78]:
for d in directories:

    #get the number key to be replaced with a well name
    orig_name, replace_key = d.split('/')[-1].split('_')
    print(orig_name, replace_key)
    
    #get the dilution factor part
    dil_num = orig_name.split(' ')[-1]
    print(dil_num)
    
    #break off the end of the directory, so we can rename that part
    parent = d.split('/')[:-1]
    #rebuild the parent
    parent_tree = ''
    for p in parent:
        parent_tree = parent_tree + p + '/'
    
    #make the new name
    new_file = parent_tree + rename[replace_key] + ' ' + time + ' ' + dil_num
    
    print(new_file)
    
    os.rename(d, new_file)

43.5 1
43.5
C:/Users/geeze/Desktop/43.5/A1 23 43.5
43.5 10
43.5
C:/Users/geeze/Desktop/43.5/C2 23 43.5
43.5 11
43.5
C:/Users/geeze/Desktop/43.5/C3 23 43.5
43.5 12
43.5
C:/Users/geeze/Desktop/43.5/C4 23 43.5
43.5 13
43.5
C:/Users/geeze/Desktop/43.5/D1 23 43.5
43.5 14
43.5
C:/Users/geeze/Desktop/43.5/D2 23 43.5
43.5 15
43.5
C:/Users/geeze/Desktop/43.5/D3 23 43.5
43.5 16
43.5
C:/Users/geeze/Desktop/43.5/D4 23 43.5
43.5 17
43.5
C:/Users/geeze/Desktop/43.5/E1 23 43.5
43.5 18
43.5
C:/Users/geeze/Desktop/43.5/E2 23 43.5
43.5 19
43.5
C:/Users/geeze/Desktop/43.5/E3 23 43.5
43.5 2
43.5
C:/Users/geeze/Desktop/43.5/A2 23 43.5
43.5 20
43.5
C:/Users/geeze/Desktop/43.5/E4 23 43.5
43.5 21
43.5
C:/Users/geeze/Desktop/43.5/F1 23 43.5
43.5 22
43.5
C:/Users/geeze/Desktop/43.5/F2 23 43.5
43.5 23
43.5
C:/Users/geeze/Desktop/43.5/F3 23 43.5
43.5 24
43.5
C:/Users/geeze/Desktop/43.5/F4 23 43.5
43.5 25
43.5
C:/Users/geeze/Desktop/43.5/G1 23 43.5
43.5 26
43.5
C:/Users/geeze/Desktop/43.5/G2 23 43.5
43.5 27
43.5
C

# rename all the 'composite.tif' files with descriptive names so they can be put together for later, and so their information shows up in title bar when open in imageJ

In [5]:
#refresh tuples of interest after changes above

tuples_of_interest = get_tuples_of_interest(directory)

In [6]:
len(tuples_of_interest)

96

In [8]:
for folder in tuples_of_interest:

    #get the directory path
    dir_path = folder[0]

    #get the new name you want for the composite file
    dirname = dir_path.split('/')[-1]

    if '_' in dirname:
        name_elements = dirname.split('_')
        
    time = name_elements[0]
    iptg = name_elements[1].split('-')[0]
    sal = name_elements[1].split('-')[1]
    dil = name_elements[2].strip('x')

    filename = '/composite.tif'

    #the existing file to rename
    orig_path = dir_path + filename

    #its new name
    new_path = dir_path + '/t{}i{}s{}d{}_merge.tif'.format(time, iptg, sal, dil)

    #some folders dont have standardized data and shouldn't be used, exclude them
    try:
        os.rename(orig_path, new_path)
    except FileNotFoundError:
        print('no composite.tif found in folder {}'.format(well_time))

# remove the merged renamed images from their subdirectories into their parent timepoint directories

In [40]:
#refresh tuples of interest after changes above

tuples_of_interest = get_tuples_of_interest(directory)

In [41]:
for t in tuples_of_interest:
    tree = t[0]
    
    files = t[2]
    
    m = [f for f in files if '_merge' in f]
    
    
    try:
        orig_name = tree + '/' + m[0]

        ##make the new name, one directory up from orig
        #break off the end of the directory, so we can rename that part
        break_off = tree.split('/')[:-1]
        #rebuild the parent
        one_up = ''
        for p in break_off:
            one_up = one_up + p + '/'

        new_name = one_up + m[0]

        os.rename(orig_name, new_name)
    
    except IndexError:
        print('could not do it for {}'.format(tree))